# Przetwarzanie języka naturalnego - lab2
## Mateusz Kocot

In [1]:
import os

from elasticsearch import Elasticsearch

import pandas as pd

from IPython.display import display, HTML

In [2]:
DATA_DIR = '../ustawy'

In [3]:
es = Elasticsearch("http://localhost:9200")

# Task 3

In [4]:
synonym_filter = {
    "type": "synonym",
    "synonyms": [
        "kpk => kodeks postępowania karnego",
        "kpc => kodeks postępowania cywilnego",
        "kk => kodeks karny",
        "kc => kodeks cywilny"
    ]
}

analyzer = {
    "default": {
        "type": "custom",
        "tokenizer": "standard", # i.
        "filter": [
            "synonym_filter",    # ii.
            "morfologik_stem",   # iii.
            "lowercase"          # iv.
        ]
    }
}

index_settings = {
    "analysis": {
        "filter": {
            "synonym_filter": synonym_filter
        },
        "analyzer": analyzer
    }
}

# Task 4

In [5]:
index_name = "legislative_acts"

if es.indices.exists(index=index_name):
    es.indices.delete(index=index_name)

es.indices.create(index=index_name, settings=index_settings)

ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'legislative_acts'})

# Task 5

In [6]:
for file_name in os.listdir(DATA_DIR):
    with open(f'{DATA_DIR}/{file_name}', 'r', encoding='UTF-8') as file:
        bill = file.read()
    es.create(index=index_name, id=file_name, document={"content": bill})

# Task 6

In [7]:
response_6 = es.count(
    index=index_name,
    query={
        "match": {
            "content": "ustawa"
        }
    }
)
# response_6

In [8]:
result_6 = response_6['count']
print('Number of bills containing the word ustawa in any form:', result_6)
print('                               Number of all the files:', len(os.listdir(DATA_DIR)))

Number of bills containing the word ustawa in any form: 1178
                               Number of all the files: 1179


All files but one contain the word ustawa in any form.

# Task 7

In [9]:
response_7 = es.termvectors(
    index=index_name,
    id="1993_599.txt",
    fields=["content"],
    term_statistics=True
)
# response_7

In [10]:
result_7 = response_7['term_vectors']['content']['terms']['ustawa']['ttf']
print('Number of occurences of ustawa in any form:', result_7)

Number of occurences of ustawa in any form: 24934


# Task 9

In [11]:
response_9 = es.count(
    index=index_name,
    query={
        "match_phrase": {
            "content": "kodeks postępowania cywilnego"
        }
    }
)
# response_9

In [12]:
result_9 = response_9['count']
print('Number of bills containing "kodeks postępowania cywilnego" in any form:', result_9)

Number of bills containing "kodeks postępowania cywilnego" in any form: 99


# Task 10

In [13]:
response_10 = es.count(
    index=index_name,
    query={
        "match_phrase": {
            "content": {
                "query": "wchodzi w życie",
                "slop": 2
            }
        }
    }
)
# response_10

In [14]:
result_10 = response_10['count']
print('Number of bills containing "wchodzi w życie" in any form (slop=2):', result_10)

Number of bills containing "wchodzi w życie" in any form (slop=2): 1174


# Task 11

In [15]:
response_11 = es.search(
    index=index_name,
    query={
        "match": {
            "content": "konstytucja"
        }
    },
    size=10
)
# response_11

In [16]:
scores = [(file_data['_id'], file_data['_score']) for file_data in response_11['hits']['hits']]

scores_df = pd.DataFrame(scores, columns=['file name', 'score'])
result_11 = scores_df.sort_values(by='score', ascending=False)

print('10 documents most relevant for "konstytucja":')
result_11

10 documents most relevant for "konstytucja":


,file name,score
0,1997_629.txt,6.869184
1,2000_443.txt,6.663479
2,1997_604.txt,6.632288
3,1996_350.txt,6.627395
4,1997_642.txt,6.252282
5,2001_23.txt,6.056855
6,1996_199.txt,5.926714
7,1999_688.txt,5.848894
8,2001_1082.txt,5.465344
9,1997_681.txt,5.465344


# Task 12

In [17]:
response_12 = es.search(
    index=index_name,
    query={
        "match": {
            "content": "konstytucja"
        }
    },
    size=10,
    highlight={
        "fields": {
            "content": {}
        },
        "number_of_fragments": 3
    }
)
# response_12

In [18]:
for file_data in response_12['hits']['hits']:
    display(HTML(f'<h3>{file_data["_id"]} (score: {file_data["_score"]:0.2f})</h3>'))
    for excerpt in file_data['highlight']['content']:
        print(excerpt)
        print('----------------------------------------------------------------------')

o zmianie ustawy konstytucyjnej o trybie przygotowania
           i uchwalenia <em>Konstytucji</em> Rzeczypospolitej
----------------------------------------------------------------------
W ustawie  konstytucyjnej z  dnia 23 kwietnia 1992 r. o trybie przygotowania i 
uchwalenia <em>Konstytucji</em>
----------------------------------------------------------------------
Do zgłoszenia projektu <em>Konstytucji</em> załącza się wykaz 
                obywateli popierających zgłoszenie
----------------------------------------------------------------------


umowy międzynarodowej i nie wypełnia przesłanek określonych w art. 89
     ust. 1 lub art. 90 <em>Konstytucji</em>
----------------------------------------------------------------------
międzynarodowej lub załącznika nie
     wypełnia przesłanek określonych w art. 89 ust. 1 lub art. 90 <em>Konstytucji</em>
----------------------------------------------------------------------
co do zasadności wyboru
  trybu ratyfikacji umowy międzynarodowej, o którym mowa w art. 89 ust. 2
  <em>Konstytucji</em>
----------------------------------------------------------------------


Jeżeli Trybunał Konstytucyjny wyda orzeczenie o sprzeczności celów partii 
   politycznej z <em>Konstytucją</em>
----------------------------------------------------------------------
Jeżeli Trybunał Konstytucyjny wyda orzeczenie o sprzeczności z <em>Konstytucją</em>
   celów lub działalności
----------------------------------------------------------------------
Ciężar udowodnienia niezgodności z <em>Konstytucją</em> spoczywa
                na wnioskodawcy, który w tym
----------------------------------------------------------------------


Za naruszenie <em>Konstytucji</em> lub ustawy, w związku z zajmowanym
              stanowiskiem lub w zakresie
----------------------------------------------------------------------
W zakresie określonym w art. 107 <em>Konstytucji</em> odpowiedzialność przed
           Trybunałem Stanu ponoszą
----------------------------------------------------------------------
Członkowie Rady Ministrów ponoszą odpowiedzialność przed Trybunałem
           Stanu za naruszenie <em>Konstytucji</em>
----------------------------------------------------------------------


wnioskami o:
             1) stwierdzenie zgodności ustaw i umów międzynarodowych z
               <em>Konstytucją</em>
----------------------------------------------------------------------
stwierdzenie zgodności przepisów prawa wydawanych przez
               centralne organy państwowe, z <em>Konstytucją</em>
----------------------------------------------------------------------
ratyfikowanymi
               umowami międzynarodowymi i ustawami,
             4) stwierdzenie zgodności z <em>Konstytucją</em>
----------------------------------------------------------------------


W Dzienniku Ustaw Rzeczypospolitej Polskiej, zwanym dalej "Dziennikiem
  Ustaw", ogłasza się:
   1) <em>Konstytucję</em>
----------------------------------------------------------------------
akty prawne dotyczące:
   1) stanu wojny i zawarcia pokoju,
   2) referendum zatwierdzającego zmianę <em>Konstytucji</em>
----------------------------------------------------------------------
ministra, któremu Sejm wyraził wotum nieufności,
     h) powoływania lub odwoływania na określone w <em>Konstytucji</em>
----------------------------------------------------------------------


2c i art. 9-11 ustawy konstytucyjnej z dnia 23 kwietnia 
1992 r. o trybie przygotowania i uchwalenia <em>Konstytucji</em>
----------------------------------------------------------------------
Prezydent Rzeczypospolitej Polskiej zarządza poddanie <em>Konstytucji</em> pod referendum
   w trybie określonym
----------------------------------------------------------------------
Przyjęcie w referendum <em>Konstytucji</em> następuje wówczas, gdy opowiedziała 
   się za nią większość biorących
----------------------------------------------------------------------


postępowania w sprawie wykonywania inicjatywy
ustawodawczej przez obywateli, o której mowa w art. 118 ust. 2 <em>Konstytucji</em>
----------------------------------------------------------------------
Projekt ustawy nie może dotyczyć spraw, dla których <em>Konstytucja</em>
Rzeczypospolitej Polskiej zastrzega wyłączną
----------------------------------------------------------------------
Projekt ustawy wniesiony do Marszałka Sejmu powinien odpowiadać wymogom
  zawartym w <em>Konstytucji</em> i Regulaminie
----------------------------------------------------------------------


Krajowa Rada Sądownictwa, zwana dalej "Radą", realizuje cele określone w
  <em>Konstytucji</em> Rzeczypospolitej
----------------------------------------------------------------------
) podejmuje uchwały w sprawach wystąpienia do Trybunału Konstytucyjnego o
     zbadanie zgodności z <em>Konstytucją</em>
----------------------------------------------------------------------


Rzecznik Praw Dziecka, zwany dalej Rzecznikiem, stoi na straży praw dziecka
  określonych w <em>Konstytucji</em>
----------------------------------------------------------------------
uroczyście, że przy wykonywaniu powierzonych mi obowiązków
     Rzecznika Praw Dziecka dochowam wierności <em>Konstytucji</em>
----------------------------------------------------------------------
